<!-- PROFILE LINK -->

<h1 style = "text-align:center; font-size: 30px">CHAT USING SOCKETS</h1>
<a href ="https://github.com/moriones2001">
<img src="https://avatars.githubusercontent.com/u/100123414?v=4" style="float:left;vertical-align:centre" width="110" height="110" title = "Iván García Moriones 21907762">
<a href ="https://github.com/moriones2001?tab=repositories">
<img src="https://media-exp1.licdn.com/dms/image/C4D0BAQEu_Aa76fcmPw/company-logo_200_200/0/1630998679429?e=2159024400&v=beta&t=2feC9GG7RWCWizTqsK5HzDWB1TL2DOQuAVwnMf6FFvk" title = "M41" alt="UE" width="110" height="110" style="float:right;vertical-align:centre">

IVÁN GARCÍA MORIONES

Actividad Individual 1

Clase Servidor

In [ ]:
#Importamos librerias necesarias 
import socket
import threading
import sys
import pickle
import os

#Existen metodos deprecated, así que con esta libreria los ocultamos de la consola
import warnings
warnings.filterwarnings("ignore")

class Servidor():

	def __init__(self, host=socket.gethostname(), port=int(input("Que puerto quiere usar ? Deben ser los primeros 5 dígitos del expediente "))):
		#Lista de conexiones y nicknames que se conectan
		self.clientes = []
		self.nicknames = []
		print('\nSu IP actual es : ',socket.gethostbyname(host))
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(), '\n\tTotal Hilos activos en este punto del programa =', threading.active_count())
		self.s = socket.socket()
		self.s.bind((str(host), int(port)))
		self.s.listen(30)
		self.s.setblocking(False)
		threading.Thread(target=self.aceptarC, daemon=True).start()
		threading.Thread(target=self.procesarC, daemon=True).start()
        #Puesta de hilos en modo demonio, tanto aceptar como procesar
        
        #Cuando el usuario inserte un '1' se cierra el socket y por lo tanto el servidor
		while True:
			msg = input('\n << SALIR = 1 >> \n')
			if msg == '1':
				print(" **** Me piro vampiro; cierro socket y mato SERVER con PID = ", os.getpid())
				self.s.close()
				sys.exit()
			else: pass
            
    #Función en la que se acepta la conexión entre servidor y cliente, se levanta hilo 
	def aceptarC(self):
		print('\nHilo ACEPTAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
			try:
				conn, addr = self.s.accept() # Conexión via ip(host) y puerto de libreria socket
				print(f"\nConexion aceptada via {addr}") 
				conn.setblocking(False)
				self.clientes.append(conn) #Añadir cliente al array de clientes
				#Primer mensaje: Recuperamos NICK del cliente
				data = conn.recv(32).decode()
				if data: 
					nickname = data
					print("NICK conectado: ", nickname)
					self.nicknames.append(nickname)

			except: pass

	def procesarC(self):
		print('\nHilo PROCESAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
			if len(self.clientes) > 0:
				for c in self.clientes:
					try:
						data = c.recv(32)
						if data: self.broadcast(data,c)
                        #Si hay data, crearemos un fichero, pasando el parámetro 'a' para que sobreescriba, con el nombre solicitado por Christian.
						with  open('u21907762AI1.txt','a') as f:
                        #Podríamos escribir primero 'Historial de conversación:' para separar distintas conversaciones
                        #f.write('Historial de conversaciones:')
							f.write(pickle.loads(data) + '\n') #Utilización de pickle.loads para trasncribri bytes a string

					except: pass

    #Mostrar los clientes conectados y los nicknames, mostrando de quien es el mensaje
	def broadcast(self, msg, cliente):
		for c in self.clientes:
			print("Clientes conectados Right now = ", len(self.clientes),' ',self.nicknames,'\tMENSAJE DE: ',pickle.loads(msg))
			try:
				if c != cliente: 
					#print(pickle.loads(msg))
					c.send(msg)

			except: self.clientes.remove(c)
		
arrancar = Servidor() 

Clase Cliente

In [ ]:
#Importamos librerias necesarias
import threading
import sys
import socket
import pickle
import os

#Existen metodos deprecated, así que con esta libreria los ocultamos de la consola
import warnings
warnings.filterwarnings("ignore")

#Creación clase cliente
class Cliente():
	#nickname=""
    #Pedir datos al cliente
	def __init__(self, host=input("Introduzca la IP del servidor :  "),port=int(input("Introduzca el PUERTO del servidor,tus primeros 5 dígitos del expediente: ")),nickname=input("Introduzca un nickname, que será tu correo de la universidad: ")):
		self.s = socket.socket()
		self.s.connect((host, int(port)))
		#self.s.connect(('192.168.1.62', int('666')))  #Conexion rapida para pruebas
		
		#Primer mensaje: Enviamos NICK al servidor
		self.nickname=nickname
		self.s.send(nickname.encode())

		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tTotal Hilos activos en este punto del programa =', threading.active_count())
		threading.Thread(target=self.recibir, daemon=True).start()
		
        
        #Salir y matar socket con el '1' 
		while True:
			#msg = input('\nEscriba texto ?   ** Enviar = ENTER   ** Salir Chat = 1: ')
			msg = input('\nNiCKNAME: ' + self.nickname + '\tEscriba texto ?   ** Enviar = ENTER   ** Salir Chat = 1\n')
			if msg != '1' : self.enviar(msg)
			else:
				print(" **** Me piro vampiro; cierro socket y mato al CLIENTE con PID = ", os.getpid())
				self.s.close()
				sys.exit()

    # Función para recibir datos de otros clientes           
	def recibir(self):
		print('\nHilo RECIBIR con ID =',threading.currentThread().getName(), '\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
        #probar si hay data y en caso afirmativo transcribir a string con pickle loads y mostrar de quien es y para quien va el mensaje         
			try:
				data = self.s.recv(32)
				if data: print(f'PARA:{self.nickname} \tDESDE: {pickle.loads(data)}')

			except: pass
    #Enviar mensajes mostrando el nickname antes que el mensaje con f"".
	def enviar(self, msg):
		mensaje = f"{self.nickname}: {msg}"
		self.s.send(pickle.dumps(mensaje))  #Envio mensaje a la otra parte en bytes
		#self.s.send(pickle.dumps(msg))

arrancar = Cliente()

MEMORIA (A PARTE DE COMENTARIOS DEL CÓDIGO)

Se nos pide modificar un chat con la librería socket para que cumpla una serie de requisitos:
1. Poder instanciar un número de 6 clientes ingresando la IP y el puerto, siendo este los 5 primeros dígitos del nº expediente. 
2. Cada vez que un cliente se conecte, este pueda y deba elegir un nickname que será vuestro correo de la universidad y que deberá ser mostrado cada vez que escriba un texto. 
3. Que el servidor, imprima y muestre la lista de usuarios ( nicknames) conectados al mismo.
4. Que el servidor almacene el historial de chat de todos los usuarios conectados durante la sesión y lo guarde en un archivo que se deberá llamar uvuestroExp.txt 

Para cada requisito:
1. Pedir al usuario mediante teclado 'input' sus primeros 5 dígitos del expediente y abrir 6 ventanas de PowerShell para los 6 clientes. (+1 de servidor)
2. Pedir en clase cliente un nickname al usuario y en la def enviar añadir el nickname del cliente.
3. Para este punto tras pedir nickname se lo enviamos al servidor y jugamos con el dumps y loads para enviar y recibir bytes y transcribir a string. Guardamos mediante 'append' los nicknames al array de nicknames del servidor.
4. Cuando se procesa la información en el servidor, si hay data: Creamos un fichero con el formato pedido por tí y y pasamos parámetro 'a' para reescribir fichero y se guarde todo el historial de todas las conexiones. Utilizo pickle loads porque recibimos bytes y hay que guardar un string. Meto el salto de línea para que salga cada mensaje en una línea. 

Para el trabajo he empleado el uso de videos en youtube como páginas del tipo de stack overflow 

    